In [1]:
!pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

  Cloning https://github.com/huggingface/transformers (to revision 21fac7abba2a37fae86106f87fcf9974fd1e3830) to /tmp/pip-req-build-tblkaah1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tblkaah1
  Running command git rev-parse -q --verify 'sha^21fac7abba2a37fae86106f87fcf9974fd1e3830'
  Running command git fetch -q https://github.com/huggingface/transformers 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Running command git checkout -q 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9677030 sha256=7acdee371cd8c10724bf4f67295ab8dd60cd9fef108582fd23d66c7b006f3aa7
  Stored in directory: /root/.cache/pip/

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [3]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [4]:
from PIL import Image
import requests

In [7]:
image = Image.open("example.png")
message= [
    {
        "role" : "user" ,
        "content" : [
            {
                "type" : "image",
            } ,
            {
                "type" : "text",
                "text": "what is written in this screenshot?"
            }
        ]
    }
]

text_prompt= processor.apply_chat_template(message, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("cuda")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)



In [8]:
print(output_text)

['The screenshot shows a set of instructions for creating a login page for a new website. The objectives and constraints are as follows:\n\n**Objectives:**\n1. Use ids "usernameMessage" and "passwordMessage" for the respective span tags to show "Username is Incorrect" and "Password is Incorrect" when the values entered are incorrect or empty.\n2. Change the color of the Login Button with id "login-btn" to Hex value #3b82f6.\n3. Ensure the Application Shows "Login successful!" when Username is "admin" and Password is "Dolphin" and Login button is clicked. The message should be displayed in a paragraph tag with id "message". The paragraph tag is already provided in the HTML code.\n\n**Constraints:**\n1. Do not change the defined "name" or "id" of any element in the HTML as it will lead to the failure of the testcases.\n2. Do not remove the CDN link for jQuery (if present) as your code will not run without it.']
